In [3]:
# Importing the required packages
import igraph as ig
import random as rd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy import stats

# Specifying the figure parameters
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 18,
        }
params = {'legend.fontsize': 16,
          'legend.handlelength': 2.}
plt.rcParams.update(params)

In [4]:
# Required functions for the SIR Simulation

def max_connect(g,m):
    """randomly removes edges for nodes that have more than m connections"""
    no_edge_rm = 0
    for n in g.vs:
        if n.degree() > m:
            nlist = g.neighbors(n)
            nremove = n.degree()-m
            nlistremove = rd.sample(nlist, k=nremove)
            no_edge_rm = no_edge_rm + nremove
            for i in nlistremove:
                g.delete_edges(g.es[g.get_eid(i, n)])
    return no_edge_rm

def remove_edg(g,m):
    """randomly removes m edges from a network g"""
    edg_rm = rd.sample(range(len(g.es)), k=m)
    g.delete_edges(edg_rm)

def results(fname,iters=100,pop=10000,setseed=3,edge_per_node=2,infect_len=5,days=170,
            mitigation=False,m=8,probs_inf=[0.01,0.01,0.1,0.2,0.3,0.3,0.3,0.25,0.2,0.15,0.1,0.05,0.01,0.01],
            mitigation2=False,m2=100):
    """stores the results of the SIR model for stats/plotting"""
    time_1perc_all = [] #time when 1% of the population is infected
    tot_time_1perc_all = [] #total time for which 1% of the population is infected
    time_1hub_all = [] #time when the first hub is infected
    cl_coeffs_all = [] #clustering coefficient of network at each iteration
    S_final = []
    R_final = []
    I_final = []
    no_rm_all = []
    peak_time_all = []
    peak_perc_all = []
    for i in range(0,iters):
        time_1perc = False
        time_1hub = False
        tt = False
        tt_peak = False
        tot_time_1perc = 0.0
        I_stores = 0.0
        setseed = setseed+1
        g = ig.Graph.Barabasi(pop, edge_per_node,power=1)
        if mitigation is True:
            no_rm = max_connect(g,m) # limits the maximum number of neighbors to m
            no_rm_all.append(no_rm)
        if mitigation2 is True:
            remove_edg(g,m2) # randomly removes m2 edges from the network
        g.vs["state"] = "S"
        g.vs["duration"] = 0
        i = rd.randint(0, pop-1)
        g.vs[i]["state"] = "I"
        for time in range(days): #no. of days
            if len(g.vs.select(state_eq = "I")) > pop*0.01 and time_1perc is False:
                #stores the time that 1% of the population is infected
                time_1perc = True
                tot_time_1perc = float(time)
                time_1perc_all.append(time)
            if len(g.vs.select(state_eq = "I")) < pop*0.01 and tot_time_1perc > 0.0 and tt is False:
                #stores the total time for which 1% of the population is infected
                tot_time = time - tot_time_1perc
                tot_time_1perc_all.append(tot_time)
                tt = True
                # stores the clustering coefficient associated with that time
                cl_coeff = g.transitivity_undirected()
                cl_coeffs_all.append(cl_coeff)
            for n in g.vs.select(state_eq = "I"): #iterates through each node in the network
                if g.vs[n.index]["duration"] is 0 and len(g.neighbors(n)) > m and time_1hub is False:
                    #stores the time that the first hub is infected
                    time_1hub = True
                    time_1hub_all.append(time)
                g.vs[n.index]["duration"] += 1 #from day 0 to infect_len this node continues to infect
                day_inf = g.vs[n.index]["duration"]
                for nb in g.neighbors(n): #iterates through neighbours of that node
                    if g.vs[nb]["state"] == "S": #if node is infected...
                        r = rd.random() #random state
                        if r < probs_inf[day_inf]:
                            g.vs[nb]["state"] = "I" #change state to infected
                if g.vs[n.index]["duration"] >= rd.randrange(2,14): #after infect_len that node changes to recovered
                    g.vs[n.index]["state"] = "R"
            if time == days-1:
                S_final.append(len(g.vs.select(state_eq = "S")))
                I_final.append(len(g.vs.select(state_eq = "I")))
                R_final.append(len(g.vs.select(state_eq = "R")))
            if time_1perc is True and I_stores > len(g.vs.select(state_eq = "I")) and tt_peak is False:
                # if pop greater than 1% and previous no. of infected nodes is higher than current assume peak
                tt_peak = True
                peak_time_all.append(time)
                peak_perc_all.append(I_stores)
            I_stores = len(g.vs.select(state_eq = "I"))
    if sum(I_final) > 0:
        print("Model not run long enough!")
    with open(fname, 'wb') as f:
        pickle.dump([peak_time_all, peak_perc_all, cl_coeffs_all, tot_time_1perc_all, time_1perc_all, time_1hub_all, S_final, I_final, R_final], f)
    return no_rm_all

In [5]:
# Storing the variables as pickle files

# Scale-Free
for i in range(50,51):
    no_rm_all = results('SIR{}.pickle'.format(i),iters=100,setseed=i*1000)

# Mitigation Hubs
avg_rm = []
for i in range(50,51):
    no_rm_all = results('SIR{}_M.pickle'.format(i),iters=100,setseed=i*1000,mitigation=True,m=8)
    avg_rm = avg_rm + no_rm_all

# Mitigation Random
for i in range(50,51):
    no_rm_all = results('SIR{}_M2.pickle'.format(i),iters=100,setseed=i*1000,mitigation2=True,m2=int(np.mean(avg_rm)))